In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sumn2u/garbage-classification-v2")

print("Path to dataset files:", path)

/home/pasha/image_classification/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/pasha/.cache/kagglehub/datasets/sumn2u/garbage-classification-v2/versions/8


In [4]:
import os
import cv2
import numpy as np
import pandas as pd
import warnings
import glob

from scipy.stats import mannwhitneyu, spearmanr, kruskal
from scipy.stats import ttest_ind, pearsonr, chi2_contingency
from sklearn.preprocessing import LabelEncoder
from scipy.stats import f_oneway
from skimage.feature import graycomatrix, graycoprops
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from collections import defaultdict

# Suppress warnings for better output
warnings.filterwarnings("ignore", category=UserWarning)

classes = os.listdir(os.path.join(path, "garbage-dataset"))
print(classes)

['trash', 'glass', 'battery', 'clothes', 'metal', 'plastic', 'cardboard', 'paper', 'biological', 'shoes']


## Извлечение фичей

In [7]:
def calculate_yellow_amount(image):
    """Calculate the amount of yellow color in the image."""
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])
    yellow_mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)
    yellow_amount = cv2.countNonZero(yellow_mask)
    return yellow_amount

def calculate_silver_amount(image):
    """Calculate the amount of silver color in the image."""
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_silver = np.array([0, 0, 200])
    upper_silver = np.array([180, 25, 255])
    silver_mask = cv2.inRange(hsv_image, lower_silver, upper_silver)
    silver_amount = cv2.countNonZero(silver_mask)
    return silver_amount

def calculate_parallel_lines(image):
    """Calculate the amount of parallel lines in the image."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, 50, 150, apertureSize=3)
    
    # Detect lines using Hough Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=100)
    
    if lines is not None:
        return len(lines)  # Return the number of detected lines
    return 0  # No lines detected

def detect_cylinders(image):
    """Detect cylindrical shapes in the image."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (9, 9), 2)

    # Detect circles using Hough Transform
    circles = cv2.HoughCircles(blurred_image, cv2.HOUGH_GRADIENT, dp=1, minDist=30,
                               param1=100, param2=30, minRadius=10, maxRadius=100)

    if circles is not None:
        return circles.shape[1]  # Return the number of detected circles
    return 0  # No circles detected

def calculate_reflection(image):
    """Calculate the amount of bright areas in the image to detect reflections."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Threshold to find bright areas
    _, bright_mask = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)
    reflection_amount = cv2.countNonZero(bright_mask)
    return reflection_amount

def calculate_transparency(image):
    """
    Calculate the amount of transparent areas in the image.
    Transparent areas are identified as regions with low saturation and brightness in the HSV color space.
    """
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Define thresholds for low saturation and brightness (transparent regions)
    lower_transparent = np.array([0, 0, 0])
    upper_transparent = np.array([180, 50, 50])  # Adjust upper threshold as needed
    transparent_mask = cv2.inRange(hsv_image, lower_transparent, upper_transparent)
    transparency_amount = cv2.countNonZero(transparent_mask)
    return transparency_amount

def calculate_texture_smoothness(image):
    """Calculate the smoothness of the texture in the image."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = graycomatrix(gray_image, [1], [0], 256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    return contrast

def calculate_shininess(image):
    """Calculate the shininess of the image based on bright pixels."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    bright_pixels = np.sum(gray_image > 220)  # Count pixels with intensity greater than 220
    total_pixels = image.shape[0] * image.shape[1]
    shininess = bright_pixels / total_pixels  # Proportion of bright pixels
    return shininess

def calculate_surface_anisotropy(image):
    """
    Calculate the anisotropy of the surface using Gabor filters.
    Measures how directional the texture is.
    """
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Parameters for Gabor filter
    num_orientations = 8  # Number of orientations to test
    kernel_size = 21      # Size of the Gabor filter kernel
    sigma = 5.0           # Standard deviation of the Gaussian function
    lambd = 10.0          # Wavelength of the sinusoidal component
    gamma = 0.5           # Spatial aspect ratio
    
    responses = []
    
    # Apply Gabor filters with different orientations
    for theta in np.linspace(0, np.pi, num_orientations, endpoint=False):
        gabor_kernel = cv2.getGaborKernel((kernel_size, kernel_size), sigma, theta, lambd, gamma, 0, ktype=cv2.CV_32F)
        filtered_image = cv2.filter2D(gray_image, cv2.CV_32F, gabor_kernel)
        responses.append(np.mean(filtered_image ** 2))  # Energy of the response

    # Measure anisotropy: ratio of maximum response to mean response
    max_response = max(responses)
    mean_response = np.mean(responses)
    anisotropy = max_response / mean_response if mean_response > 0 else 0

    return anisotropy

def calculate_aspect_ratio(image):
    """
    Calculate the average width-to-height ratio of detected objects in the image.
    """
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray_image, 50, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return 0  # No objects found

    aspect_ratios = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if h > 0:  # Avoid division by zero
            aspect_ratios.append(w / h)

    if len(aspect_ratios) == 0:
        return 0

    # Return the average aspect ratio
    return sum(aspect_ratios) / len(aspect_ratios)

def calculate_whiteness(image):
    """Calculate the proportion of white pixels in the image."""
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 50, 255])
    white_mask = cv2.inRange(hsv_image, lower_white, upper_white)
    whiteness = cv2.countNonZero(white_mask) / (image.shape[0] * image.shape[1])
    return whiteness

def calculate_line_curvature(image):
    """
    Calculate the average curvature of detected edges in the image.
    High curvature may indicate folds or wavy edges, common in clothing.
    """
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, 50, 150, apertureSize=3)
    
    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return 0  # No contours found

    curvatures = []
    for contour in contours:
        if len(contour) >= 5:  # Need at least 5 points to fit an ellipse
            ellipse = cv2.fitEllipse(contour)
            major_axis = max(ellipse[1])  # Length of the major axis
            minor_axis = min(ellipse[1])  # Length of the minor axis
            
            if major_axis > 0:
                curvature = minor_axis / major_axis  # Ratio of axes approximates curvature
                curvatures.append(curvature)
    
    if len(curvatures) == 0:
        return 0

    # Return the average curvature
    return sum(curvatures) / len(curvatures)

def get_features(image):
    """
    Get a set of features (yellow amount, silver amount, parallel lines, cylindrical shapes, 
    reflections, transparency, texture smoothness, shininess, surface anisotropy, aspect ratio,
    whiteness, edge smoothness).
    """
    yellow_amount = calculate_yellow_amount(image)
    silver_amount = calculate_silver_amount(image)
    parallel_lines = calculate_parallel_lines(image)
    cylinder_count = detect_cylinders(image)
    reflection_amount = calculate_reflection(image)
    transparency_amount = calculate_transparency(image)
    texture_smoothness = calculate_texture_smoothness(image)
    shininess = calculate_shininess(image)
    surface_anisotropy = calculate_surface_anisotropy(image)
    aspect_ratio = calculate_aspect_ratio(image)
    whiteness = calculate_whiteness(image)
    line_curvature = calculate_line_curvature(image)
    return (yellow_amount, silver_amount, parallel_lines, cylinder_count, reflection_amount,
            transparency_amount, texture_smoothness, shininess, surface_anisotropy, 
            aspect_ratio, whiteness, line_curvature)

In [51]:
# Example usage with an image
test_im = cv2.imread(path + "/garbage-dataset/cardboard/cardboard_25.jpg")
anisotropy_feature = get_features(test_im)
print("Surface Anisotropy:", anisotropy_feature)

Surface Anisotropy: (15834, 284114, 140, 15, 303454, 0, np.float64(150.46217028380636), np.float64(0.8047277777777778), np.float32(2.768033), 1.0, 0.7999305555555556, 0.39463143158270836)


## Обучение классификатора

In [52]:
# Initialize lists to hold features and labels
features = []
labels = []
test_images = []  # List to hold paths of test images

# Loop through each class and extract features from images
for class_name in classes:
    class_path = os.path.join(path + "/garbage-dataset/", class_name)
    image_names = os.listdir(class_path)

    # Split the images into training and testing sets (80% train, 20% test)
    train_images, test_images_class = train_test_split(image_names, test_size=0.2, random_state=42)

    # Process training images
    for image_name in train_images:
        image_path = os.path.join(class_path, image_name)
        image = cv2.imread(image_path)

        if image is not None:
            features_list = get_features(image)
            features.append(features_list)
            labels.append(class_name)

    # Collect test images for manual testing
    for image_name in test_images_class:
        test_images.append(os.path.join(class_path, image_name))

# Convert features and labels to a DataFrame
df = pd.DataFrame(features, columns=['yellow_amount', 'silver_amount', 'parallel_lines', 'cylinder_count', 
                                     'reflection_amount', 'transparency_amount', 'texture_smoothness', 
                                     'shininess', 'surface_anisotropy', 'aspect_ratio', 
                                     'whiteness', 'line_curvature'])
df['label'] = labels


# Split the dataset into training and testing sets
X = df[['yellow_amount', 'silver_amount', 'parallel_lines', 'cylinder_count', 
        'reflection_amount', 'transparency_amount', 'texture_smoothness', 
        'shininess', 'surface_anisotropy', 'aspect_ratio', 'whiteness','line_curvature']]
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Function to classify a new image
def classify_image(image_path):
    image = cv2.imread(image_path)
    if image is not None:
        features_list = get_features(image)
        feature = np.array([features_list])  # Convert to 2D array for prediction
        prediction = classifier.predict(feature)
        return prediction[0]
    else:
        return None

Accuracy: 0.6325110689437066
              precision    recall  f1-score   support

     battery       0.67      0.60      0.64       145
  biological       0.57      0.72      0.64       138
   cardboard       0.73      0.60      0.66       311
     clothes       0.72      0.96      0.82       886
       glass       0.51      0.57      0.54       455
       metal       0.42      0.17      0.24       163
       paper       0.63      0.50      0.56       263
     plastic       0.60      0.43      0.50       327
       shoes       0.51      0.45      0.48       317
       trash       0.61      0.47      0.53       157

    accuracy                           0.63      3162
   macro avg       0.60      0.55      0.56      3162
weighted avg       0.62      0.63      0.61      3162



In [53]:
# Classifier testing (runs classifier for a specific image)
result = classify_image(path + '/garbage-dataset/metal/metal_2370.jpg') 
print("Predicted class:", result)

Predicted class: glass


In [54]:
# Classifier testing (runs classifier for the remaining images)
def test_classify(test_images, target_class_name):
    predictions = defaultdict(int)
    for test_image in test_images:
        if target_class_name in test_image:  
            predicted_class = classify_image(test_image)  
            predictions[predicted_class] += 1 
    return dict(predictions)  

for class_name in classes:
    results = test_classify(test_images, class_name)
    print("Predicted class counts for test images containing " + class_name + ':')
    for class_name, count in results.items():
        print(f"Class: {class_name}, Count: {count}")

Predicted class counts for test images containing trash:
Class: trash, Count: 87
Class: clothes, Count: 11
Class: plastic, Count: 12
Class: glass, Count: 45
Class: paper, Count: 4
Class: shoes, Count: 9
Class: metal, Count: 6
Class: battery, Count: 3
Class: cardboard, Count: 10
Class: biological, Count: 3
Predicted class counts for test images containing glass:
Class: glass, Count: 365
Class: metal, Count: 10
Class: trash, Count: 15
Class: plastic, Count: 37
Class: shoes, Count: 54
Class: clothes, Count: 75
Class: biological, Count: 18
Class: cardboard, Count: 8
Class: paper, Count: 14
Class: battery, Count: 17
Predicted class counts for test images containing battery:
Class: shoes, Count: 29
Class: glass, Count: 23
Class: battery, Count: 116
Class: biological, Count: 11
Class: trash, Count: 1
Class: clothes, Count: 4
Class: cardboard, Count: 3
Class: plastic, Count: 1
Class: metal, Count: 1
Predicted class counts for test images containing clothes:
Class: clothes, Count: 1006
Class: p

## Проверка гипотез

### Гипотеза №1 О наличие картона на картинке можно судить по скосу желтого цвета.

- Способы проверки
    - **t-критерий Стьюдента (t-test)**: Сравнение средней доли жёлтого цвета в картинках с картоном и другими классами.
    - **ANOVA (дисперсионный анализ)**: Проверка, есть ли значимые различия между группами (картон и другие классы) по количеству жёлтого цвета.
    - **Критерий Манна-Уитни (U-тест)**: Непараметрический тест для сравнения распределений двух групп (в данном случае картон и другие классы).

In [54]:
cardboard_path = os.path.join(path + "/garbage-dataset", "cardboard")
other_paths = [os.path.join(path + "/garbage-dataset", cls) for cls in classes if cls != "cardboard"]

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        image = cv2.imread(filepath)
        if image is not None: 
            images.append(image)
    return images

cardboard_images = load_images_from_folder(cardboard_path)
cardboard_yellow_amounts = [calculate_yellow_amount(img) for img in cardboard_images]

other_yellow_amounts = []
for other_path in other_paths:
    other_images = load_images_from_folder(other_path)
    other_yellow_amounts.extend([calculate_yellow_amount(img) for img in other_images])

# Проверка гипотез
# 1. t-критерий Стьюдента
t_stat, t_pvalue = ttest_ind(cardboard_yellow_amounts, other_yellow_amounts, equal_var=False)

# 2. ANOVA
anova_stat, anova_pvalue = f_oneway(cardboard_yellow_amounts, other_yellow_amounts)

# 3. Критерий Манна-Уитни
mann_stat, mann_pvalue = mannwhitneyu(cardboard_yellow_amounts, other_yellow_amounts, alternative='two-sided')

results = {
    "t-test": {"statistic": t_stat, "p-value": t_pvalue},
    "ANOVA": {"statistic": anova_stat, "p-value": anova_pvalue},
    "Mann-Whitney": {"statistic": mann_stat, "p-value": mann_pvalue},
}

for test, result in results.items():
    print(f"{test}: Statistic = {result['statistic']}, p-value = {result['p-value']}")

alpha = 0.05
hypothesis_results = {test: result['p-value'] < alpha for test, result in results.items()}

if all(hypothesis_results.values()):
    print("Гипотеза подтверждена: наличие картона связано c желтым цветом.")
else:
    print("Гипотеза не подтверждена: наличие картона не связано c желтым цветом.")

t-test: Statistic = 3.683066343249196, p-value = 0.00023621963453461276
ANOVA: Statistic = 16.865194578791744, p-value = 4.0292196417557166e-05
Mann-Whitney: Statistic = 22080315.5, p-value = 1.0126318223771204e-146
Гипотеза подтверждена: наличие картона связано c желтым цветом.


### Гипотеза №2 О наличие металла на картинке можно судить по скосу серого (серебряного) цвета.

- Способы проверки
    - **t-критерий Стьюдента (t-test)**: Сравнение средней доли серебряного цвета в картинках с метеаллом и другими классами.
    - **ANOVA (дисперсионный анализ)**: Проверка, есть ли значимые различия между группами (металл и другие классы) по количеству серебряного цвета.
    - **Критерий Манна-Уитни (U-тест)**: Непараметрический тест для сравнения распределений двух групп (в данном случае металл и другие классы).

In [9]:
metal_path = os.path.join(path + "/garbage-dataset", "metal")
other_paths = [os.path.join(path + "/garbage-dataset", cls) for cls in classes if cls != "metal"]

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        image = cv2.imread(filepath)
        if image is not None:
            images.append(image)
    return images

metal_images = load_images_from_folder(metal_path)
metal_silver_amounts = [calculate_silver_amount(img) for img in metal_images]

other_silver_amounts = []
for other_path in other_paths:
    other_images = load_images_from_folder(other_path)
    other_silver_amounts.extend([calculate_silver_amount(img) for img in other_images])

# Проверка гипотез
# 1. t-критерий Стьюдента
t_stat, t_pvalue = ttest_ind(metal_silver_amounts, other_silver_amounts, equal_var=False)

# 2. ANOVA
anova_stat, anova_pvalue = f_oneway(metal_silver_amounts, other_silver_amounts)

# 3. Критерий Манна-Уитни
mann_stat, mann_pvalue = mannwhitneyu(metal_silver_amounts, other_silver_amounts, alternative='two-sided')

results = {
    "t-test": {"statistic": t_stat, "p-value": t_pvalue},
    "ANOVA": {"statistic": anova_stat, "p-value": anova_pvalue},
    "Mann-Whitney": {"statistic": mann_stat, "p-value": mann_pvalue},
}

for test, result in results.items():
    print(f"{test}: Statistic = {result['statistic']}, p-value = {result['p-value']}")

alpha = 0.05
hypothesis_results = {test: result['p-value'] < alpha for test, result in results.items()}

if all(hypothesis_results.values()):
    print("Гипотеза подтверждена: наличие металла связано c серебряным цветом.")
else:
    print("Гипотеза не подтверждена: наличие металла не связано c серебряным цветом.")

t-test: Statistic = -2.284424004913432, p-value = 0.022401779559765453
ANOVA: Statistic = 0.5733189930562239, p-value = 0.448952071385762
Mann-Whitney: Statistic = 10996543.5, p-value = 5.262970510833023e-16
Гипотеза не подтверждена: наличие металла не связано c серебряным цветом.


### Гипотеза №3 На картинках с картоном присутствует множество параллельных линий.

- Способы проверки
    - **t-критерий Стьюдента (t-test)**: Сравнение средней доли параллельных линий в картинках с картоном и другими классами.
    - **ANOVA (дисперсионный анализ)**: Проверка, есть ли значимые различия между группами (картон и другие классы) по количеству параллельных линий.
    - **Критерий хи-квадрат**: Проверка гипотезы об отличии доли изображений с высокими количествами параллельных линий между картоном и другими классами.

In [7]:
cardboard_path = os.path.join(path + "/garbage-dataset", "cardboard")
other_paths = [os.path.join(path + "/garbage-dataset", cls) for cls in classes if cls != "cardboard"]

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        image = cv2.imread(filepath)
        if image is not None:
            images.append(image)
    return images

cardboard_images = load_images_from_folder(cardboard_path)
cardboard_features = [calculate_parallel_lines(img) for img in cardboard_images]

other_features = []
for other_path in other_paths:
    other_images = load_images_from_folder(other_path)
    other_features.extend([calculate_parallel_lines(img) for img in other_images])

# Проверка гипотез
# 1. t-критерий Стьюдента
t_stat, t_pvalue = ttest_ind(cardboard_features, other_features, equal_var=False)

# 2. ANOVA
anova_stat, anova_pvalue = f_oneway(cardboard_features, other_features)

# 3. Критерий хи-квадрат
threshold = np.percentile(cardboard_features, 75) 
cardboard_high = sum(x >= threshold for x in cardboard_features)
cardboard_low = len(cardboard_features) - cardboard_high
other_high = sum(x >= threshold for x in other_features)
other_low = len(other_features) - other_high
chi2_stat, chi2_pvalue, _, _ = chi2_contingency([[cardboard_high, cardboard_low], [other_high, other_low]])

results = {
    "t-test": {"statistic": t_stat, "p-value": t_pvalue},
    "ANOVA": {"statistic": anova_stat, "p-value": anova_pvalue},
    "Chi-square": {"statistic": chi2_stat, "p-value": chi2_pvalue},
}

for test, result in results.items():
    print(f"{test}: Statistic = {result['statistic']}, p-value = {result['p-value']}")

# Итоговая проверка гипотезы
alpha = 0.05
hypothesis_results = {test: result['p-value'] < alpha for test, result in results.items()}

if all(hypothesis_results.values()):
    print("Гипотеза подтверждена: изображения c картоном имеют больше параллельных линий.")
else:
    print("Гипотеза не подтверждена: изображения c картоном не имеют значительно больше параллельных линий.")


t-test: Statistic = -11.379057124078555, p-value = 1.9776123451846287e-29
ANOVA: Statistic = 52.405427227996164, p-value = 4.680228101392663e-13
Chi-square: Statistic = 19.34257889433447, p-value = 1.0924297310604447e-05
Гипотеза подтверждена: изображения c картоном имеют больше параллельных линий.


### Гипотеза №5 Металлические поверхности часто имеют высокую отражающую способность. 

- Способы проверки:
    - **t-критерий Стьюдента (t-test):** Сравнение средней отражающей способности на изображениях металла и других классов.
    - **Корреляция Пирсона:** Проверка линейной связи между количеством отражений и принадлежностью к классу "металл".
    - **Критерий хи-квадрат:** Проверка отличий доли изображений с высоким уровнем отражений между металлом и другими классами.

In [2]:

metal_path = os.path.join(path + "/garbage-dataset", "metal")
other_paths = [os.path.join(path + "/garbage-dataset", cls) for cls in classes if cls != "metal"]

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        image = cv2.imread(filepath)
        if image is not None:
            images.append(image)
    return images

metal_images = load_images_from_folder(metal_path)
metal_reflections = [calculate_reflection(img) for img in metal_images]

other_reflections = []
for other_path in other_paths:
    other_images = load_images_from_folder(other_path)
    other_reflections.extend([calculate_reflection(img) for img in other_images])

# 1. t-критерий Стьюдента
t_stat, t_pvalue = ttest_ind(metal_reflections, other_reflections, equal_var=False)

# 2. Корреляция Пирсона
le = LabelEncoder()
all_reflections = metal_reflections + other_reflections
all_labels = ['metal'] * len(metal_reflections) + ['other'] * len(other_reflections)
numeric_labels = le.fit_transform(all_labels)

correlation, p_value_corr = pearsonr(all_reflections, numeric_labels)

# 3. Тест хи-квадрат
reflection_binary = [1 if reflection >= 100 else 0 for reflection in all_reflections]
contingency_table = pd.crosstab(reflection_binary, numeric_labels)
chi2_stat, p_value_chi2, _, _ = chi2_contingency(contingency_table)

results = {
    "t-test": {"statistic": t_stat, "p-value": t_pvalue},
    "Pearson Correlation": {"statistic": correlation, "p-value": p_value_corr},
    "Chi-square": {"statistic": chi2_stat, "p-value": p_value_chi2},
}

for test, result in results.items():
    print(f"{test}: Statistic = {result['statistic']}, p-value = {result['p-value']}")

alpha = 0.05
hypothesis_results = {test: result['p-value'] < alpha for test, result in results.items()}

if all(hypothesis_results.values()):
    print("Гипотеза подтверждена: наличие металла связано с высоким уровнем отражений.")
else:
    print("Гипотеза не подтверждена: наличие металла не связано с высоким уровнем отражений.")


t-test: Statistic = -2.5612454595984153, p-value = 0.010464870898548239
Pearson Correlation: Statistic = 0.005891723334828736, p-value = 0.407558471373331
Chi-square: Statistic = 35.91242941824723, p-value = 2.0638790312840235e-09
Гипотеза не подтверждена: наличие металла не связано с высоким уровнем отражений.


### Гипотеза №6: Изображения с пластиком и стеклом часто имеют больше прозрачных областей.

- Способы проверки:
    - **Тест Манна-Уитни:** Сравнение распределений количества прозрачных областей на изображениях с пластиком и стеклом против других классов.
    - **Коэффициент корреляции Спирмена:** Оценка монотонной зависимости между количеством прозрачных областей и классом изображения (пластик/стекло против остальных).
    - **Тест Крускала-Уоллиса:** Сравнение медианных значений количества прозрачных областей между группами (пластик, стекло, другие классы).

In [9]:
plastic_glass_classes = ['plastic', 'glass']
transparency_data = {'plastic': [], 'glass': [], 'other': []}

for class_name in classes:
    class_path = os.path.join(path + "/garbage-dataset/", class_name)
    image_paths = glob.glob(os.path.join(class_path, "*.jpg"))  # Предположим, что файлы изображений имеют расширение .jpg

    for image_path in image_paths:
        image = cv2.imread(image_path)
        transparency = calculate_transparency(image)

        if class_name in plastic_glass_classes:
            transparency_data['plastic'].append(transparency)
            transparency_data['glass'].append(transparency)
        else:
            transparency_data['other'].append(transparency)

# 1. Тест Манна-Уитни (для пластика/стекла против остальных)
u_statistic, p_value_mw = mannwhitneyu(transparency_data['plastic'] + transparency_data['glass'], transparency_data['other'])
print(f"Mann-Whitney U test: U-statistic = {u_statistic}, p-value = {p_value_mw}")

# 2. Коэффициент корреляции Спирмена
all_transparency = transparency_data['plastic'] + transparency_data['glass'] + transparency_data['other']
labels = ['plastic_glass'] * (len(transparency_data['plastic']) + len(transparency_data['glass'])) + ['other'] * len(transparency_data['other'])

spearman_corr, p_value_spearman = spearmanr(all_transparency, [1 if label == 'plastic_glass' else 0 for label in labels])
print(f"Spearman correlation: rho = {spearman_corr}, p-value = {p_value_spearman}")

# 3. Тест Крускала-Уоллиса
kruskal_stat, p_value_kruskal = kruskal(transparency_data['plastic'], transparency_data['glass'], transparency_data['other'])
print(f"Kruskal-Wallis test: H-statistic = {kruskal_stat}, p-value = {p_value_kruskal}")

# Интерпретация гипотезы
alpha = 0.05
if p_value_mw < alpha and p_value_spearman < alpha and p_value_kruskal < alpha:
    print("Гипотеза подтверждается: пластик и стекло имеют больше прозрачных областей.")
else:
    print("Гипотеза не подтверждается: нет статистически значимых различий между классами.")

Mann-Whitney U test: U-statistic = 63373648.0, p-value = 2.0179651032763478e-82
Spearman correlation: rho = -0.12222108103161104, p-value = 5.038550422304453e-83
Kruskal-Wallis test: H-statistic = 369.849759985094, p-value = 4.876913498602153e-81
Гипотеза подтверждается: пластик и стекло имеют больше прозрачных областей.
